In [7]:
import numpy as np 
from sympy.utilities.iterables import multiset_permutations 


Field = np.array([[0,0,0],
                  [0,0,0],
                  [0,0,0],])
Winning_pos=np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 1, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 1, 1],
                     [1, 0, 0, 1, 0, 0, 1, 0, 0],
                     [0, 1, 0, 0, 1, 0, 0, 1, 0],
                     [0, 0, 1, 0, 0, 1, 0, 0, 1],
                     [1, 0, 0, 0, 1, 0, 0, 0, 1],
                     [0, 0, 1, 0, 1, 0, 1, 0, 0]] )

In [45]:
import numpy as np 
from sympy.utilities.iterables import multiset_permutations 

def l(i):
    l= np.array([i>=1,i>=2,i>=3,i>=4,i>=5,i>=6,i>=7,i>=8,i>=9])
    return l.astype(int)

def to_binary(l):
    binary_n = 0
    for i in range (len(l)):
        binary_n += l[i]*pow(2,i)
    return binary_n

def generate_states_sing( k) :
    possible_states =[]
    for i in range (k+1):
        L = l(i)
        for p in multiset_permutations(L): 
            possible_states.append(p )
    return (possible_states)

def isin_Win(x,present=1):
    for c in Winning_pos:
        d=0
        for i in range (len(c)) :
            if c[i]==1 and x[i]==present:
                d+=1
        if d==3 : 
            return True 
    return False

def verify_win(x):
    if isin_Win(x) :
        return True 
    return False

def detect_compatibility(x , o) :
    for i in range (len(x)):
        if x[i]== 1 and o[i] == 1 : 
            return False
    if verify_win(o) and verify_win(x):
        return False
    if sum(x) < sum(o) or sum(x)>sum(o)+1 :
        return False
    if verify_win(x) and sum(x) !=sum(o)+1:
        return False
    if verify_win(o) and sum(o) !=sum(x) :
        return False
    
    return True    

def merge_element(x,o):
    new_s = []
    for i in range (len(x)):
        if  o[i] == 1 : 
            new_s.append(2)
        else : 
            new_s.append(x[i])
    return new_s

def merge_states(X_possible_states , O_possible_states) : 
    General_state =[]
    for x in X_possible_states :
        for o in O_possible_states : 
            if detect_compatibility(x , o )  :  
                General_state.append(merge_element(x,o))
    return General_state
def end_states(General_state) :
    end_states=[]
    for x in General_state :
        if isin_Win(x) :
            end_states.append(x)
            continue
        if isin_Win(x,2) :
            end_states.append(x)
            continue
        if sum(x) == 13 : 
            end_states.append(x)  
    return np.array(end_states)

tic_tac_dict= {
    0 : ' ',
    1 : 'X',
    2 : 'O'
}
def display_field (x) :
    x = x.reshape(3,3)
    for i in range (len(x)):
        for j in range(len(x[i])):
            if (j!=2):
                print(tic_tac_dict[x[i][j]],end="|")
            else : 
                print(tic_tac_dict[x[i][j]],end="") 
        print()   
        if (i!=2) : 
            for g in range(6):
                print('_',end='')
        print()
        
def reward (x):  
    if isin_Win(x) :
        return 1000
    if isin_Win(x,2):
        return -1000
    else :
        return -1
    
def Generate_rewards (General_state):
    for i in range (len(General_state)) :  
        General_state[i] = ( General_state[i] , reward(General_state[i]) )
        
        
def Actions (field):
    Actions =[]
    for i in range(len(field)):
        if field[i]==0 :
            Actions.append(i)
    return np.array(Actions)
def coding(state) : ### coding state to put it in a dict
    code=0
    for i in range(len(state)):
        code+=state[i]*pow(3,i)
    return code 

def decode(code) : 
    decoded_state=[]
    for i in range(9) : 
        decoded_state.append(code//pow(3,8-i) )
        code = code%pow(3,8-i)
    return np.array(decoded_state[::-1])


def X_O_states(General_state) :
    X_states = []
    O_states = []
    for state in General_state : 
        num_x=0
        num_o=0
        for n in state :
            if n==1 :
                num_x+=1
            if n==2 : 
                num_o+=1
        if num_x == num_o :
            X_states.append(state)
        else : 
            O_states.append(state)
    return np.array(X_states) , np.array(O_states)

#### Random X politic
## Makaynch for actions for final states 
import random

def Generate_random_politic (States):
    politic ={}
    for state in States : 
        if len((Actions(state))) !=0:
            politic[ coding(state)]=random.choice (Actions(state))
    return politic 




def quality( action , policy , state ):
    qual = 0 
    state[action]= 1
    if isin_Win(state) or (0 not in state ) :
        return reward(state)
    qual +=reward(state)
    if Actions(state) != [] : 
        for x in Actions(state):
            new_state = state.copy()   
            new_state[x] = 2 
            if isin_Win(state,2) or (0 not in state ): 
                return reward(state) 
            qual += ( 1/len(Actions(state)) ) *quality(policy[coding(new_state )] ,  policy , new_state ) 

    return qual 


def policy_iteration (policy):
    new_policy = policy.copy()
    improv = 0
    i= 0
    for state in X_states.copy() : 
        i+=1
        qa_ini = quality ( new_policy[coding(state) ]   , new_policy , state.copy()) 
        max_q = qa_ini
        max_ac = new_policy[coding(state) ] 
        for a in Actions(state) :
            if quality ( a , new_policy , state.copy()) > max_q :
                max_q = quality ( a , new_policy , state.copy()) 
                max_ac = a
        improv = max_q - qa_ini
        new_policy[coding(state)] = max_ac
    print("Iteration needed : ",i)
    return new_policy

def value_iteration( policy ):
    new_policy = policy.copy()
    k=0
    V0 = {}
    V1 = {}
    for state in X_states.copy() : 
        V0[coding(state) ] = 0 ## arbitrary initialisation
    while 1 : 
        k=k+1 
        for state in X_states.copy() : 
            ma_val = 0
            for x in Actions(state):
                new_state = state.copy()
                new_state[x] = 1
                if isin_Win(new_state) or (0 not in new_state ) :
                    continue
                val =0
                for y in Actions(new_state) : 
                    new_state2 = new_state.copy()
                    new_state2[y]=2
                    if isin_Win(new_state2,2) or (0 not in new_state2 ): 
                        continue
                    val += 1/len(Actions(new_state)) * ( reward(new_state2)  + 1 * V0[coding(new_state2)])
                if val>ma_val : 
                    ma_val = val 
            V1[coding(state)]= ma_val 
        ## checking stop criteria 
        breakk=0
        for state in X_states.copy() :
            if V0[coding(state)] > V1[coding(state)] :
                breakk=1
        if breakk ==1:
            break 
    for state in X_states.copy() :
        ma_val = 0
        ma_actions = new_policy[coding(state )]
        for x in Actions(state):
                new_state = state.copy()
                new_state[x] = 1
                if isin_Win(new_state) or (0 not in new_state ) :
                    continue
                val =0
                for y in Actions(new_state) : 
                    new_state2 = new_state.copy()
                    new_state2[y]=2
                    if isin_Win(new_state2,2) or (0 not in new_state2 ): 
                        continue
                    val += 1/len(Actions(new_state)) * ( reward(new_state2)  + 1 * V0[coding(new_state2)]) #t
                if val>ma_val :
                    ma_actions = x
                    ma_val = val 
        new_policy[coding(state )] = ma_actions


In [29]:
X_possible_states =  generate_states_sing( 5)
O_possible_states =  generate_states_sing( 4)

General_state = merge_states(X_possible_states , O_possible_states)

In [30]:
for x in end_states(General_state):
    print()
    display_field(x)


 | | 
______
 |O|O
______
X|X|X


 | | 
______
O| |O
______
X|X|X


 | | 
______
O|O| 
______
X|X|X


 | |O
______
 | |O
______
X|X|X


 | |O
______
 |O| 
______
X|X|X


 | |O
______
O| | 
______
X|X|X


 |O| 
______
 | |O
______
X|X|X


 |O| 
______
 |O| 
______
X|X|X


 |O| 
______
O| | 
______
X|X|X


 |O|O
______
 | | 
______
X|X|X


O| | 
______
 | |O
______
X|X|X


O| | 
______
 |O| 
______
X|X|X


O| | 
______
O| | 
______
X|X|X


O| |O
______
 | | 
______
X|X|X


O|O| 
______
 | | 
______
X|X|X


 | |O
______
 |O|X
______
O|X|X


O| | 
______
O| |X
______
O|X|X


O|O|O
______
 | |X
______
 |X|X


 |O| 
______
 |O|X
______
X|O|X


O|O|O
______
 | |X
______
X| |X


O| | 
______
 |O|X
______
X|X|O


O|O|O
______
 | |X
______
X|X| 


O| | 
______
O|X| 
______
O|X|X


O|O|O
______
 |X| 
______
 |X|X


O|O|O
______
 |X| 
______
X| |X


 | |O
______
 |X|O
______
X|X|O


O|O|O
______
 |X| 
______
X|X| 


O| | 
______
O|X|X
______
O| |X


O|O|O
______
 |X|X
______
 | |X


O| | 
______


In [31]:
for s in X_O_states(General_state)[0]:
    display_field(s)

 | | 
______
 | | 
______
 | | 

 | | 
______
 | | 
______
 |O|X

 | | 
______
 | | 
______
O| |X

 | | 
______
 | |O
______
 | |X

 | | 
______
 |O| 
______
 | |X

 | | 
______
O| | 
______
 | |X

 | |O
______
 | | 
______
 | |X

 |O| 
______
 | | 
______
 | |X

O| | 
______
 | | 
______
 | |X

 | | 
______
 | | 
______
 |X|O

 | | 
______
 | | 
______
O|X| 

 | | 
______
 | |O
______
 |X| 

 | | 
______
 |O| 
______
 |X| 

 | | 
______
O| | 
______
 |X| 

 | |O
______
 | | 
______
 |X| 

 |O| 
______
 | | 
______
 |X| 

O| | 
______
 | | 
______
 |X| 

 | | 
______
 | | 
______
X| |O

 | | 
______
 | | 
______
X|O| 

 | | 
______
 | |O
______
X| | 

 | | 
______
 |O| 
______
X| | 

 | | 
______
O| | 
______
X| | 

 | |O
______
 | | 
______
X| | 

 |O| 
______
 | | 
______
X| | 

O| | 
______
 | | 
______
X| | 

 | | 
______
 | |X
______
 | |O

 | | 
______
 | |X
______
 |O| 

 | | 
______
 | |X
______
O| | 

 | | 
______
 |O|X
______
 | | 

 | | 
______
O| |X
______
 | | 

 | |O
____

In [32]:
X_states , O_states =  X_O_states(General_state)
random_X_politic = Generate_random_politic (X_states)
random_O_politic = Generate_random_politic (O_states)

In [33]:
import time
start_time = time.time()

Best_policy = policy_iteration( random_X_politic)

seconds = time.time() - start_time
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))


C:\Users\jamal\AppData\Local\Temp/ipykernel_201196/1877623388.py:170: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if Actions(state) != [] :


Iteration needed :  2739
Time Taken: 00:00:08


In [ ]:
start_time = time.time()

Best_policy_valueiter = value_iteration( random_X_politic)

seconds = time.time() - start_time
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))


In [34]:
random_X_politic

{0: 7,
 10935: 5,
 8019: 2,
 7047: 7,
 6723: 2,
 6615: 1,
 6579: 1,
 6567: 7,
 6563: 4,
 15309: 6,
 3645: 2,
 2673: 8,
 2349: 8,
 2241: 4,
 2205: 1,
 2193: 0,
 2189: 3,
 13851: 4,
 5103: 4,
 1215: 1,
 891: 3,
 783: 0,
 747: 7,
 735: 0,
 731: 5,
 13365: 3,
 4617: 1,
 1701: 3,
 405: 2,
 297: 0,
 261: 0,
 249: 3,
 245: 4,
 13203: 7,
 4455: 0,
 1539: 1,
 567: 6,
 135: 8,
 99: 5,
 87: 6,
 83: 6,
 13149: 1,
 4401: 0,
 1485: 7,
 513: 0,
 189: 2,
 45: 7,
 33: 0,
 29: 5,
 13131: 6,
 4383: 8,
 1467: 0,
 495: 0,
 171: 3,
 63: 4,
 15: 0,
 11: 5,
 13125: 4,
 4377: 8,
 1461: 2,
 489: 2,
 165: 5,
 57: 2,
 21: 0,
 5: 2,
 13123: 3,
 4375: 1,
 1459: 8,
 487: 3,
 163: 7,
 55: 6,
 19: 6,
 7: 3,
 10692: 1,
 10368: 0,
 9396: 1,
 10260: 5,
 9288: 6,
 8964: 5,
 10224: 4,
 9252: 1,
 8928: 0,
 8820: 5,
 10212: 3,
 9240: 4,
 8916: 5,
 8808: 0,
 8772: 4,
 10208: 3,
 9236: 1,
 8912: 1,
 8804: 5,
 8768: 5,
 8756: 5,
 12150: 4,
 11826: 5,
 7938: 3,
 11718: 1,
 7830: 2,
 7506: 5,
 11682: 4,
 7794: 1,
 7470: 7,
 7362:

In [35]:
Best_policy

{0: 4,
 10935: 2,
 8019: 4,
 7047: 0,
 6723: 6,
 6615: 0,
 6579: 4,
 6567: 2,
 6563: 1,
 15309: 4,
 3645: 4,
 2673: 4,
 2349: 0,
 2241: 4,
 2205: 4,
 2193: 8,
 2189: 4,
 13851: 4,
 5103: 0,
 1215: 7,
 891: 8,
 783: 4,
 747: 8,
 735: 0,
 731: 4,
 13365: 4,
 4617: 8,
 1701: 4,
 405: 6,
 297: 2,
 261: 4,
 249: 3,
 245: 2,
 13203: 2,
 4455: 0,
 1539: 1,
 567: 7,
 135: 6,
 99: 1,
 87: 2,
 83: 2,
 13149: 6,
 4401: 4,
 1485: 4,
 513: 6,
 189: 6,
 45: 0,
 33: 4,
 29: 4,
 13131: 4,
 4383: 4,
 1467: 4,
 495: 0,
 171: 0,
 63: 4,
 15: 8,
 11: 4,
 13125: 4,
 4377: 2,
 1461: 4,
 489: 4,
 165: 0,
 57: 4,
 21: 4,
 5: 4,
 13123: 4,
 4375: 4,
 1459: 4,
 487: 2,
 163: 2,
 55: 8,
 19: 3,
 7: 6,
 10692: 4,
 10368: 2,
 9396: 6,
 10260: 0,
 9288: 6,
 8964: 6,
 10224: 4,
 9252: 6,
 8928: 6,
 8820: 6,
 10212: 4,
 9240: 6,
 8916: 6,
 8808: 6,
 8772: 6,
 10208: 1,
 9236: 6,
 8912: 6,
 8804: 6,
 8768: 6,
 8756: 6,
 12150: 0,
 11826: 5,
 7938: 7,
 11718: 4,
 7830: 7,
 7506: 7,
 11682: 4,
 7794: 7,
 7470: 7,
 7362:

In [21]:
for field , value in Best_policy.items()  :
    print('State : ')
    display_field(decode(field) )
    print('Value : ', value)
    print('\n\n')

State : 
 | | 
______
 | | 
______
 | | 

Value :  4



State : 
 | | 
______
 | | 
______
 |O|X

Value :  0



State : 
 | | 
______
 | | 
______
O| |X

Value :  4



State : 
 | | 
______
 | |O
______
 | |X

Value :  0



State : 
 | | 
______
 |O| 
______
 | |X

Value :  2



State : 
 | | 
______
O| | 
______
 | |X

Value :  6



State : 
 | |O
______
 | | 
______
 | |X

Value :  4



State : 
 |O| 
______
 | | 
______
 | |X

Value :  6



State : 
O| | 
______
 | | 
______
 | |X

Value :  7



State : 
 | | 
______
 | | 
______
 |X|O

Value :  4



State : 
 | | 
______
 | | 
______
O|X| 

Value :  4



State : 
 | | 
______
 | |O
______
 |X| 

Value :  6



State : 
 | | 
______
 |O| 
______
 |X| 

Value :  6



State : 
 | | 
______
O| | 
______
 |X| 

Value :  6



State : 
 | |O
______
 | | 
______
 |X| 

Value :  4



State : 
 |O| 
______
 | | 
______
 |X| 

Value :  6



State : 
O| | 
______
 | | 
______
 |X| 

Value :  4



State : 
 | | 
______
 | | 
______
X| |O

Value 

CHALLENGE POLICY ITERATION :

In [26]:

initial_field = np.zeros(9)
print("Tutoriel \n The assigned movements are this way \n[0,1,2]\n[3,4,7]\n[8,9,10]")
Random_talks =["Oh you look smart ", "That was a bad move ","You really suck I see","Damn you got me there "]
Losing_talks =["Lucky boy ", "Not again "]
Winning_talks =["GG EZ"]
while 1 :
    display_field(initial_field)
    print("The AI will play now ...")
    time.sleep(2)
    initial_field[Best_policy[coding(initial_field)]] = 1
    display_field(initial_field)
    if isin_Win(initial_field, 1) :
        print( random.choice (Winning_talks ) ) 
        display_field(initial_field)
        break 
    if (0 not in initial_field) :
        display_field(initial_field)
        break 
    print("Which move you want to play :")
    x = (int)(input())
    initial_field[x] = 2
    if isin_Win(initial_field,2) :
        display_field(initial_field)
        print( random.choice (Losing_talks ) ) 
        break 
    else : 
        display_field(initial_field)
        print( random.choice (Random_talks ) ) 
    

Tutoriel 
 The assigned movements are this way 
[0,1,2]
[3,4,7]
[8,9,10]
 | | 
______
 | | 
______
 | | 

The AI will play now ...
 | | 
______
 |X| 
______
 | | 

Which move you want to play :
0
O| | 
______
 |X| 
______
 | | 

That was a bad move 
O| | 
______
 |X| 
______
 | | 

The AI will play now ...
O|X| 
______
 |X| 
______
 | | 

Which move you want to play :
7
O|X| 
______
 |X| 
______
 |O| 

You really suck I see
O|X| 
______
 |X| 
______
 |O| 

The AI will play now ...
O|X|X
______
 |X| 
______
 |O| 

Which move you want to play :
3
O|X|X
______
O|X| 
______
 |O| 

Damn you got me there 
O|X|X
______
O|X| 
______
 |O| 

The AI will play now ...
O|X|X
______
O|X| 
______
X|O| 

GG EZ
O|X|X
______
O|X| 
______
X|O| 

